# Итераторы

Язык Python является высокоуровневым языком программирования.
В нём абсолютно всё является объектом.
В связи с этим открывается ряд новых возможностей, среди которых несколько иной подход к реализации циклов.

В отличие от других языков программирования в Python цикл `for` работает **только** с _итерируемыми_ сущностями.
Он не работает, например, с целочисленными счётчиками напрямую, как это происходит в C++, к примеру.

```{important}
Концепция итератора - это одна из ключевых особенностей Python.
```

_Итерируемый_ объект - это любой объект, реализующий _протокол итератора_.
Этот протокол лежит в основе любого _итерируемого_ типа: списка, кортежа, словаря, множества, очереди,стека, генератора и др.

```{important}
В Python вы встретите как итерируемые (`Iterable`), так и объекты типа `Sequence`, т.е. объекты, поддерживающие индексацию.
При этом `Sequence` является частным случаем итерируемого объекта, т.е. для индексируемых объектов справедливо всё, что справедливо и для итерируемых сущностей.
Например, в цикле `for` вы можете обходить как список (поддерживает индексацию), так и множество (не поддерживает индексацию).
Пример ниже.
```

In [26]:
from termcolor import cprint

# Список поддерживает индексацию
a = [1, 2, 3, 4, 3, 1]
print("a[1] =", a[1])

a[1] = 2


In [27]:
# Но множество - нет
b = set(a)
try:
    print(b[1])
except TypeError as ex:
    # Попытка обращения по индексу приводит к такому исключению
    cprint("TypeError:", "red", end=" ")
    print(ex, "(множество не индексируется)")

TypeError: 'set' object is not subscriptable (множество не индексируется)


In [28]:
# Однако и список, и множество могут
# использоваться в цикле for,
# т.к. реализуют протокол итератора
print("Список:")
for x in a:
    print(x, end=" ")
print("\nМножество:")
for s in b:
    print(s, end=" ")

Список:
1 2 3 4 3 1 
Множество:
1 2 3 4 

## Примеры стандартных функций, возвращающих итерируемый объект

Одна из наиболее часто используемых функций, которая возвращает итерируемый объект - это `range`:

In [29]:
for i in range(5):
    print(i, end=" ")

0 1 2 3 4 

```{important}
Это принципиально иной подход к реализации цикла `for` в языке.
Хоть `i` в данном случае похож на простой счётчик - это не так.
`range` возвращает **итерируемый объект**.

Ни с какими целыми числами цикл `for` в Python (напрямую) не работает, в отличие, например, от цикла `for` языка C++: `for (int i = 0; i < 5; ++i)`.
Целое же значение **генерируется** или получается при каждой итерации цикла при применении встроенной функции `next` к итерируемому объекту (подробности см. далее).
```

```{important}
Главное преимущество такого подхода - гарантированное исключение ошибки выхода за границы итерируемой сущности (массива, списка и пр.), что является одной из серьёзнейших проблем в низкоуровневых языках.
```

С помощью `range` можно создать список, например, записав _списковое выражение_, по сути своей являющееся построением списка из генератора (см. раздел [о генераторах](generators)):

In [30]:
# Либо так (это не путь Python)
a = []
for x in range(5):
    a.append(x*x)
# Либо списковым выражением (путь Python)
b = [x*x for x in range(5)]
print(f"a = {a}, b = {b}")

a = [0, 1, 4, 9, 16], b = [0, 1, 4, 9, 16]


```{note}
Второй способ, к тому же, гораздо быстрее.
```

Ещё один пример - функция `enumerate`.
Будучи применённой к переданному итерируемому объекту, она возвращает новую итерируемую сущность, которая при итерировании возвращает пару `(индекс, элемент)`.
Для начала о том, **как делать не стоит**:

In [31]:
# Пусть есть некоторый список
cubes = [x**3 for x in range(5)]
# Способ C++, но не Python
for i in range(len(cubes)):
    print(f"cubes[{i}] = {cubes[i]}", end="; ")

cubes[0] = 0; cubes[1] = 1; cubes[2] = 8; cubes[3] = 27; cubes[4] = 64; 

А вот как делают в Python:

In [32]:
for i, x in enumerate(cubes):
    print(f"cubes[{i}] = {x}", end="; ")

cubes[0] = 0; cubes[1] = 1; cubes[2] = 8; cubes[3] = 27; cubes[4] = 64; 

```{note}
Обратите внимание, что `enumerate` возвращает кортеж из двух элементов.
В данном случае этот кортеж сразу же распаковывался в две переменные `i` и `x`.
```

Ещё один пример - функция `zip`, которая помогает в одном цикле обходить сразу несколько колекций.
Допустим, есть три списка **произвольной длины**: `a`, `b` и `c`.
Нам нужно вывести сумму соответствующих элементов.
Вот способ низкоуровневых языков:

In [33]:
a = [1, 2]          # 2 элемента (самый короткий)
b = [-3, -7, 4, 0]  # 4 элемента
c = [2, -1, -4]     # 3 элемента
for i in range(len(a)):
    print(a[i] + b[i] + c[i])

0
-6


Но почему в `range` передана длина именно `a`, а не любого другого массива?
В данном случае `a` оказался самым коротким из массивов.
Другие массивы не обошлись полностью, но это не так страшно.

```{important}
Гораздо хуже было бы, окажись `a` самым длинным из всех.
**Тогда индекс `i` вышел бы за границы остальных (коротких) массивов, а это привело бы к исключению и концу работы интерпретатора**.
```

Проверять равенство длин всех массивов перед циклом?
Плохое решение.
Почему?
Функция `len` каждый раз пересчитывает длину, проходя массив полностью.
Представьте, что наши три массива содержали бы по миллиону и более элементов, и вопрос отпадёт сам собой.

Функция `zip` лишена указаных недостатков.
Если переданные ей массивы имеют разную длину, то возвращать результат она будет до тех пор, пока не закончится самый короткий список (кортеж, массив и др.).
Вот пример:

In [34]:
a = [1, 2, 3, 4]    # 4 элемента
b = [-3, -7]        # 2 элемента (cамый короткий)
c = [2, -1, -4]     # 3 элемента
for ai, bi, ci in zip(a, b, c):
    print(ai + bi + ci)

0
-6


Короткий список - `b` - имел длину 2, поэтому в результате вывелось 2 числа.
При этом мы не задумывались, какой из массивов короткий, чья длина должна быть передана для обхода по индексу.

```{note}
Функция `zip` принимает неограниченное число аргументов.
```